# GFPGAN Benchmark Notebook (stub)

This notebook benchmarks available backends with dry-run or small samples. It is a lightweight scaffold with TODOs to expand.

In [ ]:
# 1) Reload Dependencies and State
import importlib, os, random
import numpy as np

try:
    import gfpp
    from gfpp.core import registry
    from gfpp.core import orchestrator
    importlib.reload(registry)
    importlib.reload(orchestrator)
except Exception as e:
    print('Import note:', e)

random.seed(0)
np.random.seed(0)
STATE = {}  # TODO: rehydrate cached results if present

In [ ]:
# 2) Configuration and Parameters
from datetime import datetime
cfg = {
    'input': os.environ.get('GFPP_INPUT', 'assets/gfpgan_logo.png'),
    'output_dir': os.environ.get('GFPP_OUT', 'results_bench'),
    'backends': os.environ.get('GFPP_BACKENDS', 'gfpgan,codeformer,restoreformerpp').split(','),
    'metrics': os.environ.get('GFPP_METRICS', 'off'),
    'dry_run': os.environ.get('GFPP_DRY', '1') == '1',
}
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
os.makedirs(cfg['output_dir'], exist_ok=True)
cfg

In [ ]:
# 3) Logging Setup
import logging
log_path = os.path.join(cfg['output_dir'], 'bench.log')
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s %(message)s')
fh = logging.FileHandler(log_path)
fh.setLevel(logging.INFO)
fh.setFormatter(logging.Formatter('%(asctime)s %(levelname)s %(message)s'))
logging.getLogger().addHandler(fh)
logging.info('Benchmark started')
log_path

In [ ]:
# 4) Extend Core Functions (bench harness)
from typing import Dict, Any, List
def list_available_backends() -> Dict[str, bool]:
    try:
        from gfpp.core.registry import list_backends
        return list_backends(include_experimental=False)
    except Exception:
        return {}

def run_cli(input_path: str, backend: str, out_dir: str, *, dry: bool = True, metrics: str = 'off') -> int:
    """Run gfpup via subprocess.
    Returns exit code.
    Example: run_cli('assets/gfpgan_logo.png','gfpgan','out',dry=True)
    """
    import subprocess
    os.makedirs(out_dir, exist_ok=True)
    args = ['gfpup','run','--input', input_path, '--output', out_dir, '--backend', backend, '--metrics', metrics]
    if dry:
        args.append('--dry-run')
    return subprocess.call(args)

list_available_backends()

In [ ]:
# 5) Add Unit Tests (pytest) — minimal inline checks
def _test_list_backends_nonempty():
    b = list_available_backends()
    assert isinstance(b, dict)

_test_list_backends_nonempty()
print('Tests passed (inline).')

In [ ]:
# 6) Quick Benchmarks (timeit)
import time
import pandas as pd
rows = []
for be in cfg['backends']:
    t0 = time.time()
    rc = run_cli(cfg['input'], be, os.path.join(cfg['output_dir'], f'{be}_{timestamp}'), dry=cfg['dry_run'], metrics=cfg['metrics'])
    rows.append({'backend': be, 'exit_code': rc, 'sec': time.time()-t0})
df = pd.DataFrame(rows)
df

In [ ]:
# 7) Profiling (cProfile) — optional
import cProfile, pstats, io
pr = cProfile.Profile()
pr.enable()
_ = run_cli(cfg['input'], cfg['backends'][0], os.path.join(cfg['output_dir'], f'prof_{timestamp}'), dry=cfg['dry_run'], metrics=cfg['metrics'])
pr.disable()
s = io.StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats('cumtime')
ps.print_stats(10)
print(s.getvalue().splitlines()[:20])

In [ ]:
# 8) Data Validation and Error Handling — demonstration
assert os.path.exists(cfg['input']), f'Missing input: {cfg['input']}'
print('Input exists')

In [ ]:
# 9) Visualization of Results — placeholder
import matplotlib.pyplot as plt
plt.figure(figsize=(4,2))
_ = plt.plot([r['sec'] for r in rows], marker='o')
plt.title('Runtime (sec) per backend')
plt.tight_layout()
plt.show()

In [ ]:
# 10) Persist Artifacts and Outputs
csv_path = os.path.join(cfg['output_dir'], f'bench_{timestamp}.csv')
df.to_csv(csv_path, index=False)
csv_path, os.path.getsize(csv_path)

In [ ]:
# 11) Optional: CLI Entrypoint from Notebook — stub
# TODO: write a tiny wrapper to call gfpup with parameters from cfg
print('Done.')